In [1]:
import torch
import numpy as np
from scipy.io.wavfile import write
from silero_vad import get_speech_timestamps, read_audio, save_audio
import csv
import os

In [2]:
import sys
sys.path.append("C:/Users/ASUS/Desktop/FYP/Codes/whisper/Fast Whisper/faster-whisper/faster_whisper")  # Replace with actual path

from faster_whisper import WhisperModel

In [3]:
MODEL_PATH = "C:/Users/ASUS/Desktop/FYP/Codes/whisper/Fast Whisper/ctranslate2_model_4"

# Initialize Whisper model
model = WhisperModel(MODEL_PATH, device="cuda", compute_type="float32",num_workers=4)

In [4]:
def save_audio_segment(segment, sample_rate, file_name):
    """Save audio segment as a .wav file."""
    # Convert PyTorch tensor to NumPy array
    if isinstance(segment, torch.Tensor):
        segment = segment.numpy()
    # Save the audio segment
    write(file_name, sample_rate, segment)

In [ ]:
def split_audio_using_vad(audio_path, output_folder):
    """Split audio using Silero VAD and process near 10-second speech segments."""
    
    # Load Silero VAD model
    modelVAD, utils = torch.hub.load('snakers4/silero-vad', 'silero_vad', force_reload=True)
    get_speech_timestamps, *_ = utils

    # Read the audio file
    audio = read_audio(audio_path, sampling_rate=16000)

    # Run VAD to get speech timestamps
    speech_timestamps = get_speech_timestamps(audio, modelVAD, sampling_rate=16000)

    output_tsv_path = f"{output_folder}/transcriptions.tsv"
    
    min_duration = 5 * 16000  
    current_segment = []  
    accumulated_length = 0 
    i=262
    for timestamp in speech_timestamps:
        start, end = timestamp['start'], timestamp['end']
        segment = audio[start:end]  
        segment_length = end - start

        # Accumulate segments until we reach ~10s
        current_segment.append(segment)
        accumulated_length += segment_length

        if accumulated_length >= min_duration or timestamp == speech_timestamps[-1]: 
            # Merge accumulated segments
            merged_segment = torch.cat(current_segment, dim=0)  
            audio_np = merged_segment.cpu().numpy()

            # Transcribe
            segments, _ = model.transcribe(audio_np, vad_filter=False)
            transcription = " ".join(seg.text for seg in segments)

            print("Transcription:", transcription)
            output_file_name = f"{i+1}.wav"
            output_path = f"{output_folder}/{output_file_name}"
            save_audio_segment(merged_segment, 16000, output_path)
            i+=1
            # Save transcription
            with open(output_tsv_path, "a", newline="", encoding="utf-8") as tsv_file:
                writer = csv.writer(tsv_file, delimiter="\t")
                writer.writerow([output_file_name,transcription])

            # Reset buffer for next segment
            current_segment = []
            accumulated_length = 0

   


In [6]:
audio_path = "C:/Users/ASUS/Desktop/FYP/Codes/whisper/testF8.flac"
output_folder = "C:/Users/ASUS/Desktop/FYP/Codes/whisper/AugmentResultMeasure"
split_audio_using_vad(audio_path, output_folder)

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to C:\Users\ASUS/.cache\torch\hub\master.zip


Transcription: අපගේ ගුරු දේවෝතම කල්‍යාණ මිත්‍ර ගෞරවනය පින්වත් ලොකු ස්වාමීන් වහන්සේට



KeyboardInterrupt

